In [ ]:
import ast
import json
import numpy as np
import pandas as pd
import pathlib
import swifter

from sklearn.metrics import accuracy_score, f1_score


DATA_DIR = pathlib.Path("data")
EMOJI_MAPPING = {
    "❤": 0,
    "😍": 1,
    "😂": 2,
    "💕": 3,
    "🔥": 4,
    "😊": 5,
    "😎": 6,
    "✨": 7,
    "💙": 8,
    "😘": 9,
    "📷": 10,
    "🇺🇸": 11,
    "☀": 12,
    "💜": 13,
    "😉": 14,
    "💯": 15,
    "😁": 16,
    "🎄": 17,
    "📸": 18,
    "😜": 19
}

In [ ]:
def get_results(df: pd.DataFrame) -> pd.DataFrame:
    df["chatgpt_final"] = df["chatgpt_final"].swifter.apply(lambda x: ast.literal_eval(x))
    
    y_true = df["annotation"].to_numpy()
    
    gpt_answer = df["chatgpt_final"].to_list()
    y_pred_top1 = np.array([answer[0] for answer in gpt_answer])
    y_pred_top3 = np.array(
        [
            y_true[i] if y_true[i] in answer else answer[0]
            for i, answer in enumerate(gpt_answer)
        ]
    )
    
    f1_macro = f1_score(y_true=y_true, y_pred=y_pred_top1, average="macro")
    f1_micro = f1_score(y_true=y_true, y_pred=y_pred_top1, average="micro")
    acc_top1 = accuracy_score(y_true=y_true, y_pred=y_pred_top1)
    acc_top3 = accuracy_score(y_true=y_true, y_pred=y_pred_top3)
    
    return pd.DataFrame(
        [
            {"metric": "F1_macro", "value": f1_macro},
            {"metric": "F1_micro", "value": f1_micro},
            {"metric": "acc_top1", "value": acc_top1},
            {"metric": "acc_top3", "value": acc_top3},
        ]
    )

In [ ]:
#df = pd.read_csv(DATA_DIR.joinpath("Ewaluacja ChatGPT - zadania - TweetEval - emojis.csv"))
df = pd.read_excel('ChatGPT Evaluation v2.0.xlsx', sheet_name='21.TweetEmoji')
df = df.drop(df[df["chatgpt_final"] == "[-1]"].index)
get_results(df)

Pandas Apply:   0%|          | 0/1664 [00:00<?, ?it/s]

,metric,value
0,F1_macro,0.181922
1,F1_micro,0.295072
2,acc_top1,0.295072
3,acc_top3,0.506611
